In [23]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [26]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [27]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [28]:
from collections import defaultdict

In [29]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [30]:
len(hashes), len(documents)

(947, 948)

In [31]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [32]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [33]:
import json

In [28]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [29]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [30]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [31]:
import os

In [32]:
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

In [33]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [34]:
from tqdm.auto import tqdm

In [35]:
results = {}

In [36]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [1]:
import pickle

In [38]:
with open('results.bin', 'wb') as f:
    pickle.dump(results, f)

In [2]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [3]:
results

{'c02e79ef': '```json\n[\n  "What is the specific date and time when the course officially begins?",\n  "How can I add the course schedule to my personal calendar to stay updated?",\n  "Is there a registration link I need to use before the course starts, and where can I find it?",\n  "Where should I go to receive important announcements about the course?",\n  "Do I need to join any communication platforms like Slack or Telegram for this course, and how?"\n]\n```',
 '1f6520ca': '```json\n[\n    "What prior knowledge or skills do I need before enrolling in this course?",\n    "Are there any specific tools or technologies I should be familiar with before starting?",\n    "Does the course require experience with programming languages or frameworks?",\n    "Where can I find the list of requirements to take this course?",\n    "Is there a recommended background or preparation guide for this course?"\n]\n```',
 '7842b56a': '```json\n[\n    "Is it possible to enroll in the course after the off

In [16]:
cleaned_results = {}

for key, value in results.items():
    # Remove markdown fences
    #That string contains:
    #Markdown: ```json
    #Newlines \n
    #JSON array text
    cleaned = value.strip()
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()
    
    # Parse JSON string into Python list
    cleaned_results[key] = json.loads(cleaned)

In [17]:
cleaned_results

{'c02e79ef': ['What is the specific date and time when the course officially begins?',
  'How can I add the course schedule to my personal calendar to stay updated?',
  'Is there a registration link I need to use before the course starts, and where can I find it?',
  'Where should I go to receive important announcements about the course?',
  'Do I need to join any communication platforms like Slack or Telegram for this course, and how?'],
 '1f6520ca': ['What prior knowledge or skills do I need before enrolling in this course?',
  'Are there any specific tools or technologies I should be familiar with before starting?',
  'Does the course require experience with programming languages or frameworks?',
  'Where can I find the list of requirements to take this course?',
  'Is there a recommended background or preparation guide for this course?'],
 '7842b56a': ['Is it possible to enroll in the course after the official start date has already passed?',
  'What happens if I decide to join the

In [21]:
type(cleaned_results)

dict

In [18]:
cleaned_results['1f6520ca']

['What prior knowledge or skills do I need before enrolling in this course?',
 'Are there any specific tools or technologies I should be familiar with before starting?',
 'Does the course require experience with programming languages or frameworks?',
 'Where can I find the list of requirements to take this course?',
 'Is there a recommended background or preparation guide for this course?']

In [34]:
doc_index = {d['id']: d for d in documents}

In [35]:
final_results = []

for doc_id, questions in cleaned_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [36]:
import pandas as pd

In [37]:
final_results

[('What is the specific date and time when the course officially begins?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('How can I add the course schedule to my personal calendar to stay updated?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Is there a registration link I need to use before the course starts, and where can I find it?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where should I go to receive important announcements about the course?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Do I need to join any communication platforms like Slack or Telegram for this course, and how?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('What prior knowledge or skills do I need before enrolling in this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Are there any specific tools or technologies I should be familiar with before starting?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Does the course require experience with programming languages or framewor

In [38]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [39]:
df.to_csv('ground-truth-data.csv', index=False)

In [40]:
!head ground-truth-data.csv

question,course,document
What is the specific date and time when the course officially begins?,data-engineering-zoomcamp,c02e79ef
How can I add the course schedule to my personal calendar to stay updated?,data-engineering-zoomcamp,c02e79ef
"Is there a registration link I need to use before the course starts, and where can I find it?",data-engineering-zoomcamp,c02e79ef
Where should I go to receive important announcements about the course?,data-engineering-zoomcamp,c02e79ef
"Do I need to join any communication platforms like Slack or Telegram for this course, and how?",data-engineering-zoomcamp,c02e79ef
What prior knowledge or skills do I need before enrolling in this course?,data-engineering-zoomcamp,1f6520ca
Are there any specific tools or technologies I should be familiar with before starting?,data-engineering-zoomcamp,1f6520ca
Does the course require experience with programming languages or frameworks?,data-engineering-zoomcamp,1f6520ca
Where can I find the list of requirements to ta